In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install import-ipynb

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import math
import pickle
import time
import sys

import import_ipynb
import VariationalAutoencoderModel

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
importing Jupyter notebook from VariationalAutoencoderModel.ipynb
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-09-20 18:18:45.153973: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 18:18:45.271099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-20 18:18:45.271115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-20 18:18:45.290794: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-20 18:18:45.762151: W tensorflow/stream_executor/pla

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-09-20 18:18:46.234532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 18:18:46.272431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-20 18:18:46.272531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-20 18:18:46.272625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared obj

# BotIoT Data Pre-Processing

In [10]:
# Traffic information

path = '../botiot/'
all_files = glob.glob(os.path.join(path , '*.csv'))

files_list = []
for file in all_files:
    df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
    files_list.append(df)

df_traffic = pd.concat(files_list, axis = 0, ignore_index = True)
df_traffic = df_traffic.drop(columns=['pkSeqID', 'stime', 'flgs', 'flgs_number', 'saddr', 'sport', 'daddr', 'dport', 'category', 'subcategory'])
df_traffic['proto'] = df_traffic['proto'].map({'tcp': 1, 'arp': 2, 'udp': 3, 'icmp': 4, 'ipv6-icmp': 5})
df_traffic['state'] = df_traffic['state'].map({'REQ': 1, 'RST': 2, 'ACC': 3, 'CON': 4, 'INT': 5, 'URP': 6, 'FIN': 7, 'NRS': 8, 'ECO': 9, 'TST': 10, 'MAS': 11})

# Benign traffic

df_benign = df_traffic.query('attack == 0')
df_benign.drop(columns=['attack'])

# Attacks

df_attack = df_traffic.query('attack == 1')
df_attack.drop(columns=['attack'])

,proto,proto_number,pkts,bytes,state,state_number,ltime,seq,dur,mean,...,TnP_PerProto,TnP_Per_Dport,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP
0,1,1,5,770,1,3,1.528096e+09,86607,14.713629,2.669374,...,502,502,0.342776,0.326786,100,46,0.339821,0.326786,495,225
1,1,1,5,770,1,3,1.528096e+09,86608,14.713629,2.669375,...,502,502,0.342776,0.326786,100,46,0.339821,0.326786,495,225
2,1,1,5,770,1,3,1.528096e+09,86609,14.713628,2.669375,...,502,502,0.342776,0.326786,100,46,0.339821,0.326786,495,225
3,1,1,5,770,1,3,1.528096e+09,86610,14.713627,2.669375,...,502,502,0.342776,0.326786,100,46,0.339821,0.326786,495,225
4,1,1,5,770,1,3,1.528096e+09,86611,14.713627,2.674472,...,502,502,0.342776,0.326786,100,46,0.339821,0.326786,495,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668517,3,3,8,480,5,4,1.528085e+09,120562,25.001644,3.565629,...,938,938,0.319979,0.319979,54,54,0.319979,0.360424,432,432
3668518,3,3,8,480,5,4,1.528085e+09,120563,25.001644,3.565628,...,938,938,0.319979,0.319979,54,54,0.319979,0.360424,432,432
3668519,3,3,8,480,5,4,1.528085e+09,120564,25.001644,3.565628,...,938,938,0.319979,0.319979,54,54,0.319979,0.360424,432,432
3668520,3,3,8,480,5,4,1.528085e+09,120565,25.001644,3.565624,...,938,938,0.319979,0.319979,54,54,0.319979,0.360424,432,432


In [4]:
scaler = MinMaxScaler()
df_benign_norm = scaler.fit_transform(df_benign)
df_attack_norm = scaler.fit_transform(df_attack)

# Variational Autoencoder - Attack Detection

## Model

In [8]:
# Train set

len_benign = len(df_benign_norm)
len_benign_train = int(0.7 * len_benign)
X_train_VAE = df_benign_norm[:len_benign_train]
display(pd.DataFrame(X_train_VAE))

# Test set
X_benign_test = df_benign_norm[len_benign_train:]
X_attack_test = df_attack_norm
X_test_VAE = np.concatenate([X_benign_test, X_attack_test])

Y_test_VAE = np.ones(len(X_test_VAE))
Y_test_VAE[:len(X_benign_test)] = 0

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.25,0.25,0.000043,2.505802e-06,0.333333,0.142857,0.128132,0.000237,0.616220,0.000001,...,0.000127,1.688967e-07,1.839471e-08,1.000000,0.013699,5.403665e-07,0.000005,0.000027,0.000042,0.0
1,0.00,0.00,0.000128,8.631096e-06,0.333333,0.142857,0.170441,0.000267,0.749125,0.000006,...,0.000037,2.315534e-07,3.782810e-08,0.000000,0.013699,6.533948e-07,0.000006,0.000080,0.000076,0.0
2,0.50,0.50,0.000014,1.670535e-06,0.333333,0.142857,0.000000,0.000297,0.000025,0.009777,...,0.000004,1.858526e-05,2.265007e-04,0.000000,0.342466,3.912286e-03,0.033131,0.000009,0.000025,0.0
3,0.25,0.25,0.000128,6.264505e-06,0.333333,0.142857,0.171058,0.000326,0.749194,0.000048,...,0.000127,2.315322e-07,3.782464e-08,0.000000,0.000000,5.403665e-07,0.000005,0.000080,0.000076,0.0
4,0.50,0.50,0.000043,7.935040e-06,0.333333,0.142857,0.075871,0.000385,0.293650,0.019831,...,0.000061,2.362846e-07,3.860102e-08,0.035714,0.013699,6.667452e-07,0.000022,0.000027,0.000025,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0.50,0.50,0.000014,1.559166e-06,0.333333,0.142857,0.718464,0.219642,0.000004,0.001466,...,0.000524,6.040366e-05,1.483467e-03,0.166667,0.136986,2.608457e-02,0.000260,0.000258,0.000059,0.0
329,1.00,1.00,0.000000,1.392112e-07,1.000000,1.000000,0.720921,0.252231,0.000000,0.000000,...,0.055199,0.000000e+00,0.000000e+00,0.023810,0.000000,0.000000e+00,0.000000,0.000018,0.000000,0.0
330,1.00,1.00,0.000000,1.392112e-07,1.000000,1.000000,0.721095,0.252409,0.000000,0.000000,...,0.055199,0.000000e+00,0.000000e+00,0.023810,0.000000,0.000000e+00,0.000000,0.000018,0.000000,0.0
331,0.50,0.50,0.000014,1.670535e-06,0.333333,0.142857,0.722258,0.253714,0.000025,0.009760,...,0.000037,6.668552e-05,2.272289e-04,0.023810,0.109589,3.919069e-03,0.021644,0.000044,0.000059,0.0


In [6]:
# KL Loss function

def vae_loss(x, x_decoded_mean):
    # Compute the average MSE error, then scale it up (sum on all axes)
    
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean))
    
    # Compute the KL loss
    
    kl_loss = - 0.5 * K.sum(1 + z_var - K.square(z_mean) - K.square(K.exp(z_var)), axis=-1)
    
    # Return the average loss over all 
    
    total_loss = K.mean(reconstruction_loss + kl_loss) # Total_loss = reconstruction_loss + kl_loss 
    return total_loss

# (1) Reconstruction Loss - Forces the encoder to generate latent features that minimize the reconstruction error, or else is
# penalized
# (2) KL Loss - Forces the distribution generated by the encoder to be similar to the prior probability of the input vector, 
# pushing latent feature space to normality

In [7]:
# Parameters

original_dim = X_train_VAE.shape[1]
input_shape = (original_dim,)
intermediate_dim = int(original_dim / 2)
latent_dim = int(original_dim / 3)

print(X_train_VAE.shape)

epochs = 800
learning_rate = 0.0001
batch_size = 50
anomaly_threshold = 0.042

(333, 36)


In [ ]:
# Encoder

inputs, encoder, z_var, z_mean = VariationalAutoencoderModel.vae_encoder(
    input_shape, 
    intermediate_dim, 
    latent_dim, 
    VariationalAutoencoderModel.sample)

encoder.summary()

In [ ]:
# Decoder

decoder = VariationalAutoencoderModel.vae_decoder(
    intermediate_dim,
    latent_dim,
    original_dim)

decoder.summary()

In [ ]:
# VAE model

outputs = decoder(encoder(inputs))

opt = optimizers.Adam(learning_rate = learning_rate, clipvalue = 0.5)

vae_model = Model(inputs, outputs, name = 'vae_mlp')
vae_model.compile(optimizer = opt, loss = vae_loss)
vae_model.summary()

## Train

In [ ]:
train_start = time.time()

results = vae_model.fit(X_train_VAE, X_train_VAE,
                        shuffle = True,
                        epochs = epochs,
                        batch_size = batch_size)

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

In [ ]:
plt.plot(results.history['loss'])
plt.title('Loss Pattern')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper right');
plt.show()

## Test

In [ ]:
X_train_pred = vae_model.predict(X_train_VAE)
error_vector = VariationalAutoencoderModel.get_error_term(X_train_pred, X_train_VAE, _rmse=False)
error_thresh = anomaly_threshold

# Measure the error between the train set and its output and generates an error vector representing the error of each term
# Find an extreme value on the vector to use as the error threshold

In [ ]:
test_start = time.time()

X_pred_VAE = vae_model.predict(X_test_VAE)

test_end = time.time()
test_time = test_end - test_start
print("Testing time:", test_time)

# If the error of the vector is higher than the defined threshold it detects an attack

error_vector = VariationalAutoencoderModel.get_error_term(X_pred_VAE, X_test_VAE, _rmse=False)
attacks = (error_vector > error_thresh)
print(attacks)

np.count_nonzero(attacks) / len(attacks) # Percentage of attacks

In [ ]:
print(len(X_test_VAE))

## Metrics

In [ ]:
sys.stdout = open("../Results/bot_iot.txt", "a")

print(" ==== Test Bot-Iot" + "Epochs:" + str(epochs) + "LR:" + str(learning_rate) + "AT:" + str(anomaly_threshold) + "Batch:" + str(batch_size) + "====")
print("Training time:" + str(train_time))
print("Testing time:" + str(test_time))
print(classification_report(Y_test_VAE, attacks, digits = 5))